In [ ]:
!pip install streamlit shap plotly fastapi uvicorn pyngrok pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 42.1 MB/s eta 0:00:00


In [ ]:
!pip install lime shap


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=264943dcb216cfb1ad9bfb7e8c0b32084a018da57014d8079d6c430cff3d510b
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [ ]:
from pyngrok import ngrok
!ngrok authtoken "34iMBn6qULNlltUgCWvRFLkTK3g_5BQFCDsd7h92NrXJT2PRn"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
ngrok.kill()

In [ ]:
%%writefile app.py
from fastapi import FastAPI, UploadFile
import pandas as pd
from io import StringIO

app = FastAPI()
data = None

@app.post("/upload")
async def upload_csv(file: UploadFile):
    global data
    content = await file.read()
    data = pd.read_csv(StringIO(content.decode("utf-8")))
    return {"columns": list(data.columns), "rows": len(data)}

@app.get("/sample")
async def sample():
    global data
    if data is None:
        return {"error": "No data loaded"}
    return data.head(50).to_dict(orient="records")

Writing app.py


In [ ]:
from pyngrok import ngrok
import threading, os

port = 8501
public_url = ngrok.connect(port).public_url
print("🌐 Streamlit App URL:", public_url)

def run_streamlit():
    os.system(f"streamlit run dashboard.py --server.port {port}")

threading.Thread(target=run_streamlit).start()


🌐 Streamlit App URL: https://seema-ecclesiastical-dendritically.ngrok-free.dev


In [ ]:
from pathlib import Path
bg_path = Path("matrix_ai.jpg")
bg_url = bg_path.as_posix()

In [ ]:
with open("dashboard.py","w") as f:
    f.write("""
import streamlit as st
import numpy as np
import pandas as pd
import joblib

st.set_page_config(page_title="CFRP Fatigue Failure Analysis", layout="wide")

# ======== CSS STYLING ========
st.markdown('''
<style>

/* ===== Animated MATRIX green-black background ===== */
@keyframes drift {
  0%   {background-position: 0 0, 0 0, 0 0;}
  50%  {background-position: 100px 200px, -200px 100px, 300px 400px;}
  100% {background-position: 0 0, 0 0, 0 0;}
}

[data-testid="stAppViewContainer"] {
  background-color: #000 !important;
  background-image:
     radial-gradient(circle at 20% 30%, rgba(0,255,0,0.05) 0%, transparent 70%),
     radial-gradient(circle at 80% 70%, rgba(0,255,0,0.08) 0%, transparent 70%),
     repeating-linear-gradient(180deg, rgba(0,255,0,0.05) 0, rgba(0,255,0,0.05) 2px, transparent 3px, transparent 6px);
  background-size: 400px 400px, 600px 600px, 8px 12px;
  animation: drift 40s linear infinite;
  font-family: "Consolas", monospace;
  color: #00FF00;
}

/* ===== Floating glow layer ===== */
[data-testid="stAppViewContainer"]::before {
  content: "";
  position: fixed;
  top: 0; left: 0; width: 100%; height: 100%;
  background: radial-gradient(circle at 50% 50%, rgba(0,255,0,0.1) 0%, rgba(0,0,0,0.9) 80%);
  pointer-events: none;
  animation: pulse 8s ease-in-out infinite alternate;
  z-index: -1;
}
@keyframes pulse {
  from {opacity: 0.3;}
  to   {opacity: 0.6;}
}

/* ===== Control bar ===== */
.matrix-bar {
  background: rgba(0,0,0,0.9);
  border: 1px solid #00FF00;
  box-shadow: 0 0 20px #00FF00;
  border-radius: 10px;
  padding: 12px 18px;
  display: flex;
  justify-content: space-evenly;
  align-items: center;
  flex-wrap: wrap;
  position: fixed;
  top: 4%;
  left: 50%;
  transform: translateX(-50%);
  width: 94%;
  z-index: 10;
}

/* Input fields */
input {
  background-color: black !important;
  color: #00FF00 !important;
  border: 1px solid #00FF00 !important;
  border-radius: 5px;
  text-align: center;
}

/* Labels */
label {
  color: #00FF00 !important;
  font-size: 13px !important;
}

/* Predict button */
.stButton>button {
  background-color: #00FF00 !important;
  color: black !important;
  font-weight: bold;
  border-radius: 6px;
  box-shadow: 0 0 18px #00FF00;
  transition: all 0.3s;
}
.stButton>button:hover {
  background-color: black !important;
  color: #00FF00 !important;
  box-shadow: 0 0 35px #00FF00;
}

/* Title + Result box */
h1 {
  color: #00FF00 !important;
  text-align: center;
  letter-spacing: 1px;
  text-shadow: 0 0 15px #00FF00;
  margin-top: 160px;
}
.result-box {
  text-align: center;
  color: #00FF00;
  font-weight: bold;
  margin-top: 30px;
  font-size: 20px;
  text-shadow: 0 0 10px #00FF00;
}

</style>
''', unsafe_allow_html=True)

# ======== MODEL LOADING ========
MODEL_PATH = "L1_RUL_RF_pipeline.pkl"
try:
    pipeline = joblib.load(MODEL_PATH)
except:
    pipeline = None
    st.warning("⚠️ Model file 'L1_RUL_RF_pipeline.pkl' not found. Please upload it to the same folder.")

# ======== CONTROL BAR ========
st.markdown('<div class="matrix-bar">', unsafe_allow_html=True)
col1,col2,col3,col4,col5,col6,col7 = st.columns([1,1,1,1,1,1,1.3])
with col1: strain_mean = st.number_input("Strain Mean of Means (–)", value=0.01, format="%.6f")
with col2: median_strain = st.number_input("Median Strain (–)", value=0.0, format="%.6f")
with col3: pzt_amp_std = st.number_input("PZT Amplitude Std (V)", value=0.01, format="%.6f")
with col4: D_smooth = st.number_input("D_smooth (–)", value=0.0, format="%.6f")
with col5: strain_std_mean = st.number_input("Strain Std of Means (–)", value=0.0, format="%.6f")
with col6: load = st.number_input("Load (N)", value=0.01, format="%.6f")
with col7: coupon_id = st.text_input("Coupon ID", value="L1_S11")
st.markdown('</div>', unsafe_allow_html=True)

# ======== TITLE ========
st.markdown("<h1>CFRP FATIGUE FAILURE ANALYSIS</h1>", unsafe_allow_html=True)
st.caption("Neural Diagnostic Engine — Monochrome AI | True Matrix Mode")

# ======== PREDICTION ========
predict_btn = st.button("Predict RUL")
if predict_btn:
    if pipeline:
        input_data = pd.DataFrame([{
            'strain_mean_of_means': strain_mean,
            'median_strain': median_strain,
            'pzt_amplitude_std': pzt_amp_std,
            'D_smooth': D_smooth,
            'strain_std_of_means': strain_std_mean,
            'load': load
        }])
        predicted_rul = pipeline.predict(input_data)[0]
        st.markdown(f"<div class='result-box'>Predicted Remaining Useful Life (RUL): <span style='background:#00FF00;color:black;padding:4px 10px;border-radius:5px;'>{predicted_rul:.2f}</span></div>", unsafe_allow_html=True)
    else:
        st.markdown("<div class='result-box'>Model not found — unable to predict.</div>", unsafe_allow_html=True)
""")
